In [1]:
# from pymagnitude import Magnitude

In [2]:
# glove = Magnitude('data/magnitude/glove.6B.300d.magnitude')

In [3]:
# fasttext = Magnitude('data/magnitude/wiki-news-300d-1M-subword.magnitude', case_insensitive=True)

In [4]:
# print(glove.dim)
# print(fasttext.dim)

300
300


In [6]:
# print(glove[0])

('the', array([ 8.986000e-03,  4.114330e-02, -1.435200e-03, -8.849740e-02,
       -6.878300e-03,  4.563060e-02, -5.565290e-02,  4.153510e-02,
       -2.602770e-02, -3.167679e-01, -5.035510e-02,  6.259700e-03,
        1.092770e-02, -8.356000e-03, -4.182700e-03,  4.337830e-02,
       -1.449980e-02, -1.293430e-02, -2.749640e-02,  7.493200e-03,
       -3.657510e-02,  5.785510e-02,  7.585790e-02,  3.452160e-02,
       -3.347170e-02, -4.087310e-02,  4.558040e-02, -1.229030e-02,
       -8.167300e-02, -2.250550e-02,  1.809430e-02,  3.338100e-02,
       -6.383030e-02,  9.478530e-02, -1.331591e-01, -1.784500e-02,
        4.775160e-02, -3.472230e-02,  1.889610e-02,  1.604160e-02,
        2.952680e-02, -5.264220e-02, -7.514200e-03,  1.050933e-01,
        1.037114e-01,  5.617210e-02, -1.418800e-03,  9.240800e-03,
       -7.866600e-02, -5.164400e-03,  3.458330e-02,  2.118500e-03,
       -2.115840e-02, -5.094190e-02,  1.427990e-02,  5.063500e-02,
       -2.910410e-02,  6.682170e-02,  4.971250e-02,  2

In [18]:
# glove.query(["I", "read", "a", "book"])

array([[-0.02280903,  0.02282331, -0.04478139, ..., -0.02167702,
         0.00340514,  0.06512546],
       [-0.0421094 , -0.0626592 ,  0.0066932 , ...,  0.0204491 ,
        -0.0954911 , -0.0082391 ],
       [-0.0482601 ,  0.015276  , -0.0157005 , ...,  0.0096996 ,
        -0.0371193 ,  0.0480815 ],
       [ 0.0080143 , -0.0090586 ,  0.0245809 , ...,  0.0844503 ,
         0.0224759 , -0.0207853 ]], dtype=float32)

In [19]:
# fasttext.query([["I", "read", "a", "book"], ["I", "read", "a", "magazine"]])

array([[[-0.0413727,  0.0290336,  0.0096557, ..., -0.0233056,
          0.017242 , -0.0187049],
        [-0.0613015, -0.0976044,  0.0546563, ..., -0.0688883,
         -0.0051112, -0.0399671],
        [-0.0041607, -0.050058 ,  0.0164242, ..., -0.0044358,
         -0.0351897,  0.0133512],
        [-0.0275018, -0.0647214,  0.0377438, ..., -0.0004722,
         -0.0205171, -0.0230596]],

       [[-0.0413727,  0.0290336,  0.0096557, ..., -0.0233056,
          0.017242 , -0.0187049],
        [-0.0613015, -0.0976044,  0.0546563, ..., -0.0688883,
         -0.0051112, -0.0399671],
        [-0.0041607, -0.050058 ,  0.0164242, ..., -0.0044358,
         -0.0351897,  0.0133512],
        [ 0.0170235, -0.0525393,  0.1137774, ..., -0.0401786,
         -0.0365987, -0.0305128]]], dtype=float32)

In [4]:
# vectors = Magnitude(glove, fasttext)

In [5]:
# vectors.query(["I", "read", "a", "book"])

array([[-0.02280903,  0.02282331, -0.04478139, ..., -0.0233056 ,
         0.017242  , -0.0187049 ],
       [-0.0421094 , -0.0626592 ,  0.0066932 , ..., -0.0688883 ,
        -0.0051112 , -0.0399671 ],
       [-0.0482601 ,  0.015276  , -0.0157005 , ..., -0.0044358 ,
        -0.0351897 ,  0.0133512 ],
       [ 0.0080143 , -0.0090586 ,  0.0245809 , ..., -0.0004722 ,
        -0.0205171 , -0.0230596 ]], dtype=float32)

In [13]:
# vectors.query([["I", "read", "a", "book"], ["I", "read", "a", "magazine"]])

array([[[-0.02280903,  0.02282331, -0.04478139, ..., -0.0233056 ,
          0.017242  , -0.0187049 ],
        [-0.0421094 , -0.0626592 ,  0.0066932 , ..., -0.0688883 ,
         -0.0051112 , -0.0399671 ],
        [-0.0482601 ,  0.015276  , -0.0157005 , ..., -0.0044358 ,
         -0.0351897 ,  0.0133512 ],
        [ 0.0080143 , -0.0090586 ,  0.0245809 , ..., -0.0004722 ,
         -0.0205171 , -0.0230596 ]],

       [[-0.02280903,  0.02282331, -0.04478139, ..., -0.0233056 ,
          0.017242  , -0.0187049 ],
        [-0.0421094 , -0.0626592 ,  0.0066932 , ..., -0.0688883 ,
         -0.0051112 , -0.0399671 ],
        [-0.0482601 ,  0.015276  , -0.0157005 , ..., -0.0044358 ,
         -0.0351897 ,  0.0133512 ],
        [ 0.0623805 ,  0.0460008 ,  0.066375  , ..., -0.0401786 ,
         -0.0365987 , -0.0305128 ]]], dtype=float32)

In [5]:
# print(vectors.dim)
# emdim = vectors.dim

600


In [1]:
emdim = 600

In [2]:
from keras.layers import Input, TimeDistributed, LSTM, Bidirectional
from keras.models import Model
from layers import Highway, Similarity, C2QAttention, Q2CAttention, MergedContext, SpanBegin, SpanEnd, CombineOutputs
from scripts import negative_avg_log_error, BatchGenerator

Using TensorFlow backend.


In [3]:
question_input = Input(shape=(None, emdim), dtype='float32', name="question_input")
passage_input = Input(shape=(None, emdim), dtype='float32', name="passage_input")

In [4]:
# def highway_layer(value, activation="tanh", transform_gate_bias=-1.0):
#     dim = K.int_shape(value)[-1]
#     transform_gate_bias_initializer = keras.initializers.Constant(transform_gate_bias)
#     transform_gate = Dense(units=dim, bias_initializer=transform_gate_bias_initializer)(value)
#     transform_gate = Activation("sigmoid")(transform_gate)
#     carry_gate = Lambda(lambda x: 1.0 - x, output_shape=(dim,))(transform_gate)
#     transformed_data = Dense(units=dim)(value)
#     transformed_data = Activation(activation)(transformed_data)
#     transformed_gated = Multiply()([transform_gate, transformed_data])
#     identity_gated = Multiply()([carry_gate, value])
#     value = Add()([transformed_gated, identity_gated])
#     return value

In [5]:
num_highway_layers = 2

In [6]:
for i in range(num_highway_layers):
    highway_layer = Highway(name='highway_{}'.format(i))
    question_layer = TimeDistributed(highway_layer, name=highway_layer.name + "_qtd")
    question_embedding = question_layer(question_input)
    passage_layer = TimeDistributed(highway_layer, name=highway_layer.name + "_ptd")
    passage_embedding = passage_layer(passage_input)

In [7]:
# model = keras.models.Sequential()
# model.add(Dense(15, input_shape=(5, 125, 10)))
# model.summary()

In [8]:
# a = {'a': 1}
# b = a.get('b')
# print(b)
# b in a

In [9]:
# a = {'a': 1}
# b = a.pop('a')
# print(b)
# print(a)

In [10]:
encoder_layer = Bidirectional(LSTM(emdim, return_sequences=True), name='bidirectional_encoder')
encoded_question = encoder_layer(question_embedding)
encoded_passage = encoder_layer(passage_embedding)

In [11]:
similarity_matrix = Similarity(name='similarity_layer')([encoded_passage, encoded_question])

In [12]:
# import numpy as np
# a = np.random.random((600, 1))
# b = np.random.random((600, 1))
# c = np.dot(a.T, b)
# print(c)

In [13]:
context_to_query_attention = C2QAttention(name='context_to_query_attention')([similarity_matrix, encoded_passage])
query_to_context_attention = Q2CAttention(name='query_to_context_attention')([similarity_matrix, encoded_question])

In [14]:
merged_context = MergedContext(name='merged_context')([encoded_passage, context_to_query_attention, query_to_context_attention])

In [15]:
num_decoders = 1

In [16]:
modeled_passage = merged_context
for i in range(num_decoders):
    hidden_layer = Bidirectional(LSTM(emdim, return_sequences=True), name='bidirectional_decoder_{}'.format(i))
    modeled_passage = hidden_layer(modeled_passage)

In [17]:
span_begin_probabilities = SpanBegin(name='span_begin')([merged_context, modeled_passage])
span_end_probabilities = SpanEnd(name='span_end')([encoded_passage, merged_context, modeled_passage, span_begin_probabilities])

In [18]:
output = CombineOutputs(name='combine_outputs')([span_begin_probabilities, span_end_probabilities])

In [19]:
model = Model([question_input, passage_input], [output])

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
passage_input (InputLayer)      (None, None, 600)    0                                            
__________________________________________________________________________________________________
question_input (InputLayer)     (None, None, 600)    0                                            
__________________________________________________________________________________________________
highway_1_ptd (TimeDistributed) (None, None, 600)    0           passage_input[0][0]              
__________________________________________________________________________________________________
highway_1_qtd (TimeDistributed) (None, None, 600)    0           question_input[0][0]             
__________________________________________________________________________________________________
bidirectio

In [21]:
model.compile(loss=negative_avg_log_error, optimizer='adadelta', metrics='loss')

<class 'tensorflow.python.framework.ops.Tensor'>
(None, None)
(2, None, None)
(None, None, 1)
(None, None, 1)
(None,)
(None, 1)
(None, 1)


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Tensor'

In [ ]:
# import numpy as np
# a = np.random.random((10, 1))
# b = np.random.random((10, 1))
# # print(a)
# print(b[0])
# for v in a:
#     for w in b:
#         print(w)
# #         c = a*b
# #         print(c)
#         break
#     break

In [ ]:
# print(encoded_question.shape[0])

In [ ]:
# import numpy as np
# from keras.layers import concatenate, multiply, dot, Input


# def concatenate_and_multiply(context_vector, query_vector):
#     element_wise_mul = multiply([context_vector, query_vector], axis=-1)
#     concatenate_vecs = concatenate([context_vector, query_vector, element_wise_mul], axis=-1)
#     return concatenate_vecs


# a = np.random.random((10, 10, 20))
# b = np.random.random((10, 15, 20))
# v = Input(shape=(10, 20))
# w = Input(shape=(15, 20))

# kernel = np.random.random((60, ))
# concatenated_vector = concatenate_and_multiply(v, w)
# result = dot([kernel, concatenated_vector], axes=-1)
# print(result.shape)

In [ ]:
# import numpy as np 
# from keras import backend as K 

# A = np.random.random((1000,32,64,3))
# # B = np.moveaxis( A, 3, 1)
# C = np.transpose( A, (0,3,1,2))

# print(A.shape)
# print(C.shape)

# A_t = K.variable(A)
# C_t = K.permute_dimensions(A_t, (0,3,1,2))

# print(K.eval(A_t).shape)
# print(K.eval(C_t).shape)

In [ ]:
# import numpy as np 
# from keras import backend as K 
# from keras.models import Input, Model
# from keras.layers.advanced_activations import Softmax

# A = np.random.random((10,32,64)).astype('float32')

# i = Input(shape=(32,64), dtype='float32')
# o = Softmax(axis=-1)(i)
# model = Model([i], [o])
# y = model.predict([A])
# print(y)
# print(y.shape)
# print(np.sum(y, axis=-2))
# print(np.sum(y, axis=-1))
# print(np.sum(y, axis=-1).shape)
# print(np.sum(y, axis=-2).shape)

In [ ]:
# import numpy as np 
# from keras import backend as K 

# A = np.random.random((32,6))
# # B = np.moveaxis( A, 3, 1)
# # C = np.transpose( A, (0,3,1,2))
# B = np.random.random((32, 6))

# print(A.shape)
# print(B.shape)

# def mul(x):
#     A, B = x
#     print(K.int_shape(A))
#     print(K.int_shape(B))
#     return A*B

# A_t = K.variable(A)
# B_t = K.variable(B)
# # C_t = K.permute_dimensions(A_t, (0,3,1,2))

# C = K.map_fn(mul, (A_t, B_t), dtype='float32')

# y = K.eval(C)